In [1]:
import os

In [2]:
%pwd

'/Users/atatekeli/PycharmProjects/coccidiosis-classifier/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/atatekeli/PycharmProjects/coccidiosis-classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen = True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_logger: Path
    checkpoint_model_filepath: Path

In [6]:
from src.CoccidiosisClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf